# 📰 新聞轉教案生成器 - Google Colab 教學

> **這是一個規格驅動的專案**  
> 你可以使用 `specs/news-to-lesson.spec.md` 中的規格，配合 AI 工具完成開發。

## 📋 學習目標

1. 學習 AI API 整合（OpenAI/Anthropic/Gemini）
2. 掌握 Prompt Engineering 技巧
3. 實作多受眾適配功能
4. 體驗規格驅動開發

## 🚀 快速開始

本教學使用 **Google Gemini API**（免費），你也可以使用 OpenAI 或 Anthropic API。

## 步驟 1：安裝依賴套件

In [ ]:
!pip install google-generativeai requests beautifulsoup4 -q

print("✅ 依賴套件安裝完成！")

## 步驟 2：設定 API Key

### 取得免費的 Gemini API Key：
1. 前往 https://aistudio.google.com/
2. 點擊「Get API Key」
3. 複製你的 API Key
4. 貼到下方

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# 方法 1：使用 Colab Secrets（推薦）
# 在左側欄 🔑 Secrets 中新增 GEMINI_API_KEY
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ 從 Secrets 讀取 API Key")
except:
    # 方法 2：直接輸入（不安全，僅供測試）
    API_KEY = input("請輸入你的 Gemini API Key: ")

# 設定 API
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-pro')

print("✅ API 設定完成！")

## 步驟 3：查看規格和 Prompt 模板

這個專案支援三種受眾類型：

1. **企業高管 (Executive)**
   - 專業術語密度：12-15%
   - 強調 ROI 和商業價值
   - 包含企業案例

2. **成人學員 (Adult)**
   - 實用案例導向
   - 問句比例：10-15%
   - 平易近人的語氣

3. **年長者 (Senior)**
   - 淺顯易懂
   - 豐富的類比說明
   - 專業術語需解釋

In [ ]:
# Prompt 模板
PROMPTS = {
    "executive": """
你是一位商業顧問，請將以下新聞轉換為適合企業高管的教學內容。

要求：
- 專業術語密度：12-15%
- 強調 ROI 和商業價值
- 包含至少一個企業案例
- 結論先行結構

請按照以下格式輸出：

# [教案標題]

**受眾：** 企業高管

## 📋 摘要
[2-3 句話總結核心內容，強調商業價值]

## 🎯 知識點
1. [知識點 1 - 與商業決策相關]
2. [知識點 2 - 與 ROI 相關]
3. [知識點 3 - 與競爭優勢相關]

## 💡 企業案例
[提供至少一個相關企業案例]

## 🔑 關鍵要點
- [要點 1 - 可執行的商業建議]
- [要點 2 - 風險與機會分析]
- [要點 3 - 實施建議]

## 📝 結論
[總結商業影響與行動建議]

---

新聞內容：
{news_content}
""",
    
    "adult": """
你是一位實用主義教師，請將以下新聞轉換為適合成人學員的教學內容。

要求：
- 實用案例導向
- 問句比例：10-15%
- 平易近人的語氣
- 包含行動建議

請按照以下格式輸出：

# [教案標題]

**受眾：** 成人學員

## 📋 摘要
[2-3 句話總結，用日常語言]

## 🎯 知識點
1. [知識點 1 - 實用角度]
2. [知識點 2 - 生活應用]
3. [知識點 3 - 技能提升]

## 💡 實例說明
[提供貼近生活的實際案例]

## 🔑 關鍵要點
- [要點 1 - 可以立即應用的]
- [要點 2 - 常見問題解答]
- [要點 3 - 進階學習方向]

## 📝 行動建議
[具體的下一步行動]

---

新聞內容：
{news_content}
""",
    
    "senior": """
你是一位耐心的教育者，請將以下新聞轉換為適合年長者的教學內容。

要求：
- 淺顯易懂
- 豐富的類比說明
- 專業術語需解釋
- 避免複雜句型

請按照以下格式輸出：

# [教案標題]

**受眾：** 年長者

## 📋 簡單說明
[用最簡單的語言解釋這篇新聞在說什麼]

## 🎯 重點整理
1. [重點 1 - 用類比說明]
2. [重點 2 - 舉生活例子]
3. [重點 3 - 避免專業術語]

## 💡 舉例說明
[用熟悉的事物類比，例如：「就像...一樣」]

## 🔑 簡單記住
- [要點 1 - 一句話總結]
- [要點 2 - 容易記憶的]
- [要點 3 - 實用建議]

## 📝 總結
[用簡單的話總結，避免複雜概念]

---

新聞內容：
{news_content}
"""
}

print("✅ Prompt 模板載入完成！")
print(f"支援的受眾類型：{list(PROMPTS.keys())}")

## 步驟 4：實作轉換函式

In [ ]:
def convert_news_to_lesson(news_content: str, audience: str = "adult") -> str:
    """
    將新聞轉換為教案
    
    Args:
        news_content: 新聞內容
        audience: 受眾類型 (executive/adult/senior)
    
    Returns:
        教案內容（Markdown 格式）
    """
    if audience not in PROMPTS:
        raise ValueError(f"不支援的受眾類型：{audience}")
    
    # 取得 Prompt 模板
    prompt = PROMPTS[audience].format(news_content=news_content)
    
    # 調用 Gemini API
    print(f"🤖 正在生成教案（受眾：{audience}）...")
    response = model.generate_content(prompt)
    
    return response.text

print("✅ 轉換函式定義完成！")

## 步驟 5：測試轉換功能

In [ ]:
# 範例新聞
sample_news = """
AI 時代的軟體開發革命

【科技新聞】2025年10月17日

人工智慧正在徹底改變軟體開發的方式。根據最新調查，超過70%的開發者已經開始使用AI輔助工具，
如GitHub Copilot、Cursor和Windsurf等，來提升開發效率。

專家指出，傳統的開發方式需要開發者花費大量時間在撰寫程式碼、除錯和測試上。但現在，AI工具可以
在幾秒鐘內生成高品質的程式碼，讓開發者能夠專注於更高價值的工作，如需求分析、系統設計和商業邏輯。

然而，這並不意味著開發者會被取代。相反地，開發者的角色正在轉變。「AI可以告訴你怎麼做，但只有人
能決定做什麼和為什麼做。」一位資深架構師表示。

業界專家建議，開發者應該學習如何撰寫清晰的規格文件，因為這將成為與AI協作的關鍵。規格驅動開發
（SDD）方法論正在受到越來越多關注。

對於企業而言，這意味著可以更快速地推出產品，降低開發成本，同時保持高品質。對於個人開發者，
這是一個重新定位自己價值的機會。
"""

# 測試三種受眾類型
print("=" * 60)
print("測試：企業高管版")
print("=" * 60)
lesson_executive = convert_news_to_lesson(sample_news, "executive")
print(lesson_executive)
print("\n")

In [ ]:
print("=" * 60)
print("測試：成人學員版")
print("=" * 60)
lesson_adult = convert_news_to_lesson(sample_news, "adult")
print(lesson_adult)
print("\n")

In [ ]:
print("=" * 60)
print("測試：年長者版")
print("=" * 60)
lesson_senior = convert_news_to_lesson(sample_news, "senior")
print(lesson_senior)

## 步驟 6：輸入你自己的新聞

In [ ]:
# 輸入新聞內容
your_news = input("請貼上新聞內容：\n")

# 選擇受眾
print("\n選擇受眾類型：")
print("1. 企業高管 (executive)")
print("2. 成人學員 (adult)")
print("3. 年長者 (senior)")
choice = input("請輸入選項 (1/2/3): ")

audience_map = {"1": "executive", "2": "adult", "3": "senior"}
audience = audience_map.get(choice, "adult")

# 生成教案
lesson = convert_news_to_lesson(your_news, audience)
print("\n" + "=" * 60)
print("生成的教案：")
print("=" * 60)
print(lesson)

## 步驟 7：儲存教案

In [ ]:
from datetime import datetime
from google.colab import files

# 儲存教案
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"lesson_{audience}_{timestamp}.md"

with open(filename, 'w', encoding='utf-8') as f:
    f.write(lesson)

print(f"✅ 教案已儲存：{filename}")

# 下載檔案
files.download(filename)
print("📥 檔案已下載到你的電腦")

## 🎓 學習總結

### 你學到了什麼？

1. ✅ **AI API 整合**
   - 使用 Google Gemini API
   - 設定和調用 API
   - 處理 API 回應

2. ✅ **Prompt Engineering**
   - 設計結構化 Prompt
   - 針對不同受眾調整
   - 控制輸出格式

3. ✅ **多受眾適配**
   - 企業高管：專業、ROI 導向
   - 成人學員：實用、行動導向
   - 年長者：簡單、類比說明

### 進階挑戰

1. 📝 實作 URL 解析功能（從網址抓取新聞）
2. 🧪 實作驗收標準自動化檢查
3. 💰 實作成本追蹤功能
4. 🎨 建立 Web 介面

### 下一步

1. 嘗試不同的新聞主題
2. 優化 Prompt 模板
3. 比較不同 AI 模型的效果
4. 整合到實際工作流程中

---

**🎉 恭喜完成新聞轉教案生成器教學！**

更多資源：
- [完整規格文件](../specs/news-to-lesson.spec.md)
- [專案 README](../README.md)
- [SDD 學習指南](https://github.com/你的帳號/SDD-learning-guide-main)